In [247]:
import numpy as np
import pandas as pd

from os import listdir
from os.path import isfile, join

Crime and Incarceration in the USA

In [248]:
crime_incarceration = pd.read_csv('../input/prisoners-and-crime-in-united-states/crime_and_incarceration_by_state.csv')
crime_incarceration = crime_incarceration.convert_dtypes()
# combining rapes
crime_incarceration['rape'] = crime_incarceration.rape_legacy.fillna(crime_incarceration.rape_revised)
crime_incarceration.drop(columns=['includes_jails',
                                  'prisoner_count',
                                  'crime_reporting_change',
                                  'crimes_estimated',
                                  'rape_legacy',
                                  'rape_revised'], inplace=True)
# dropping federal jurisdiction
crime_incarceration = crime_incarceration[crime_incarceration.jurisdiction != 'FEDERAL']
crime_incarceration = crime_incarceration.rename(columns={'jurisdiction': 'state'})
crime_incarceration.state = crime_incarceration.state.str.title()
crime_incarceration = crime_incarceration.rename(columns={'murder_manslaughter': 'murder'})

In [249]:
# NaNs in each state data
crime_incarceration.set_index('state').isna().groupby(level=0).sum().sum(axis=1)

In [250]:
# let's look at New York
crime_incarceration[crime_incarceration.state == 'New York']

In [251]:
columns = ['year', 'state_population', 'violent_crime_total',
           'murder', 'robbery', 'agg_assault',
           'property_crime_total', 'burglary', 'larceny',
           'vehicle_theft', 'rape']

In [252]:
# filling in the missing values with mean of previous and next year 
crime_incarceration[columns] = ((crime_incarceration.groupby('state')[columns].ffill() + \
                                 crime_incarceration.groupby('state')[columns].bfill()) / 2).astype('int')

In [253]:
crime_incarceration[crime_incarceration.state == 'New York']

In [254]:
years = crime_incarceration.year.astype('str').unique()

US Mass Shootings

In [255]:
mass_shootings = pd.read_csv('../input/us-mass-shootings-last-50-years/Mass shooting data.csv')
mass_shootings['year'] = mass_shootings['Incident Date'].apply(lambda x: x.split(',')[-1].strip())
mass_shootings = mass_shootings[mass_shootings['year'].isin(years)]
mass_shootings['mass_shooting_victims'] = mass_shootings['# Killed'] + mass_shootings['# Injured']
mass_shootings.drop(columns=['Incident ID',
                             'Incident Date',
                             'City Or County',
                             'Address',
                             '# Killed',
                             '# Injured',
                             'Operations'], inplace=True)
mass_shootings.head()

In [256]:
# what years are considered?
mass_shootings['year'].value_counts()

In [257]:
mass_shootings_prev = pd.read_csv('../input/us-mass-shootings-last-50-years/Mass Shootings Dataset Ver 3.csv',
                                  encoding='latin')
# drop data with empty Location
mass_shootings_prev.dropna(subset=['Location'], inplace=True)
mass_shootings_prev['State'] = mass_shootings_prev.Location.apply(lambda x: x.split(',')[-1].strip())
mass_shootings_prev['year'] = mass_shootings_prev.Date.apply(lambda x: x.split('/')[-1].strip())
mass_shootings_prev = mass_shootings_prev[mass_shootings_prev['year'].isin(years)]
# dropping 2016 data since it has already been considered
mass_shootings_prev = mass_shootings_prev[mass_shootings_prev['year'] != '2016']
mass_shootings_prev = mass_shootings_prev[['State', 'year', 'Total victims']]
mass_shootings_prev = mass_shootings_prev.rename(columns={'Total victims': 'mass_shooting_victims'})
# change CA -> California
mass_shootings_prev = mass_shootings_prev.replace(states)
# change Washington D.C. -> District of Columbia 
mass_shootings_prev.State = mass_shootings_prev.State.replace({'Washington D.C.': 'District of Columbia'})
mass_shootings_prev.head()

In [258]:
# combining 2 mass shooting datasets
mass_shootings = pd.concat([mass_shootings, mass_shootings_prev])
mass_shootings = mass_shootings.groupby(['State', 'year']) \
                               .agg({'year':'count', 'mass_shooting_victims': 'sum'}) \
                               .rename(columns={'year': 'mass_shootings'}).reset_index()
mass_shootings = mass_shootings.rename(columns={'State': 'state'})
# dropping victims
mass_shootings.drop(columns=['mass_shooting_victims'], inplace=True)
mass_shootings.year = mass_shootings.year.astype('int')
mass_shootings.head()

Murders by method, USA, 1999-2016

The data starts since 2004

In [259]:
# addirional information on Alabama murders
alabama_murders = pd.read_csv('../input/alabama-crimes/alabama.csv', sep=';')

In [260]:
murders = pd.read_csv('../input/murders-by-method-usa-19992016/murders by type.csv')
murders = murders[murders['Year'].astype('str').isin(years)]
murders = murders.pivot(index=['Year', 'State'], columns='Type', values='By Weapon').reset_index()
murders.columns = ['year',
                   'state',
                   'murder_cutting',
                   'murder_firearms',
                   'murder_other', 
                   'murder_unarmed']
# dropping non-states USA territories
murders = murders[~murders['state'].isin(['Guam', 'U.S. Virgin Islands', 'Virgin Islands'])]
murders['total'] = murders[['murder_cutting',
                            'murder_firearms',
                            'murder_other', 
                            'murder_unarmed']].sum(axis=1)
murders = pd.concat([murders, alabama_murders])
murders.head()

In [261]:
murder_comparison = pd.merge(murders[['year', 'state', 'total']],
                             crime_incarceration[['year', 'state', 'murder']],
                             left_on=['year', 'state'],
                             right_on=['year', 'state'],
                             how='left')
murder_comparison.dropna(inplace=True)
murder_comparison[~(murder_comparison['total'] <= murder_comparison['murder'])] 

FBI Hate Crimes in USA (1991-2020)

In [262]:
hate_crimes = pd.read_csv('../input/fbi-hate-crimes-in-usa-19912020/Hate Crimes in USA 1991-2020.csv',
                          low_memory=False)
hate_crimes = hate_crimes[hate_crimes['STATE_NAME'] != 'Guam']
hate_crimes = hate_crimes[['DATA_YEAR', 'STATE_ABBR', 'STATE_NAME', 'OFFENSE_NAME']]
hate_crimes = hate_crimes[hate_crimes['DATA_YEAR'].astype('str').isin(years)]
# dropping Guam
hate_crimes = hate_crimes[hate_crimes['STATE_NAME'] != 'Guam']
# counting murders and non-murders
hate_crimes['hate_crime_murder'] = np.where(hate_crimes['OFFENSE_NAME'].str.contains('Murder and Nonnegligent Manslaughter'), 1, 0)
hate_crimes['hate_crime_other'] = np.where(hate_crimes['OFFENSE_NAME'].str.contains('Murder and Nonnegligent Manslaughter'), 0, 1)
# dropping offence name
hate_crimes.drop(columns=['OFFENSE_NAME'], inplace=True)
hate_crimes.columns = ['year', 'STATE_ABBR', 'state', 'hate_crime_murder', 'hate_crime_other']
hate_crimes.head()

In [263]:
hate_crimes = hate_crimes.groupby(['year', 'STATE_ABBR', 'state']) \
                         .agg({'hate_crime_murder': 'sum', 'hate_crime_other': 'sum'}).reset_index()

Concatenation

In [264]:
states = hate_crimes.groupby(['STATE_ABBR', 'state']).size().reset_index()
states = states[states.state != 'Guam']
states = pd.Series(states.state.values,index=states.STATE_ABBR).to_dict()
states.update({'HI': 'Hawaii'})

In [265]:
states

In [266]:
# crime_incarceration + murders
crime_incarceration = pd.merge(crime_incarceration,
                               murders.drop(columns=['total']),
                               left_on=['year', 'state'],
                               right_on=['year', 'state'],
                               how='outer')

In [267]:
# crime_incarceration + murders + mass_shootings
crime_incarceration = pd.merge(crime_incarceration,
                               mass_shootings,
                               left_on=['year', 'state'],
                               right_on=['year', 'state'],
                               how='outer')

In [268]:
# crime_incarceration + murders + mass_shootings + hate_crimes
crime_incarceration = pd.merge(crime_incarceration,
                               hate_crimes.drop(columns=['STATE_ABBR']),
                               left_on=['year', 'state'],
                               right_on=['year', 'state'],
                               how='outer')

In [269]:
crime_incarceration = crime_incarceration[~crime_incarceration['state'].isin(['National',
                                                                              'District of Columbia'])]
crime_incarceration.reset_index(drop=True, inplace=True)

In [270]:
crime_incarceration.isnull().sum(axis=0)

In [271]:
# fill mass_shootings with 0 (they did not happen)
crime_incarceration.mass_shootings = crime_incarceration.mass_shootings.fillna(0)

In [272]:
crime_incarceration = crime_incarceration.convert_dtypes()

In [273]:
columns = ['hate_crime_murder', 'hate_crime_other']

In [274]:
# no hate crime information on Hawaii
crime_incarceration[crime_incarceration[columns].isnull().any(axis=1) & 
                    (crime_incarceration.state == 'Hawaii')][['state', 'year'] + columns]

In [276]:
# filling in hate_crime_murder
fill_values = pd.concat([crime_incarceration.groupby('state')
                                            .fillna(method='ffill')
                                            .convert_dtypes()[columns[0]],
                         crime_incarceration.groupby('state')
                                            .fillna(method='bfill')
                                            .convert_dtypes()[columns[0]]], axis=1).mean(axis=1)
condition = crime_incarceration[columns[0]].isnull() & (crime_incarceration.state != 'Hawaii')
crime_incarceration.loc[condition, columns[0]] = fill_values[condition].astype('int')

In [277]:
# filling in hate_crime_other
fill_values = pd.concat([crime_incarceration.groupby('state')
                                            .fillna(method='ffill')
                                            .convert_dtypes()[columns[1]],
                         crime_incarceration.groupby('state')
                                            .fillna(method='bfill')
                                            .convert_dtypes()[columns[1]]], axis=1).mean(axis=1)
condition = crime_incarceration[columns[1]].isnull() & (crime_incarceration.state != 'Hawaii')
crime_incarceration.loc[condition, columns[1]] = fill_values[condition].astype('int')

In [278]:
columns = ['murder_cutting', 'murder_firearms', 'murder_other', 'murder_unarmed']

In [279]:
# no data on murder types in Florida
crime_incarceration[crime_incarceration[columns].isnull().any(axis=1) & \
                    ~crime_incarceration.year.isin([2001, 2002, 2003])]

In [280]:
crime_incarceration.dtypes

In [281]:
national_murders = pd.read_csv('../input/murders-by-method-usa-19992016/murders by type.csv')
national_murders = national_murders[national_murders.Year.isin([2001, 2002, 2003]) & \
                                    (national_murders.State == 'National')]
national_murders = national_murders.pivot(index=['Year', 'State'],
                                          columns='Type',
                                          values='By Weapon').reset_index()
national_murders.columns = ['year',
                            'state',
                            'murder_cutting',
                            'murder_firearms',
                            'murder_other', 
                            'murder_unarmed']

In [289]:
national_murders

In [290]:
# counting total for each column
national = crime_incarceration.groupby(['year']).sum().reset_index()
national['state'] = 'National'

In [294]:
# incerting values for types of murders
columns = ['murder_cutting', 'murder_firearms', 'murder_other', 'murder_unarmed']
national.loc[national.year.isin([2001, 2002, 2003]), columns] = national_murders.loc[:, columns] 

In [298]:
crime_incarceration = pd.concat([national, crime_incarceration])
crime_incarceration.drop(columns=['violent_crime_total', 'property_crime_total'], inplace=True)

In [310]:
crime_incarceration

In [315]:
# incerting total column
columns = ['murder', 'robbery', 'agg_assault', 'rape', # violent crimes
           'burglary', 'larceny', 'vehicle_theft', # property crimes
           'mass_shootings', 'hate_crime_murder', 'hate_crime_other']
crime_incarceration['total'] = crime_incarceration[columns].sum(axis=1).astype('int')

In [317]:
crime_incarceration.reset_index(inplace=True, drop=True)

In [320]:
crime_incarceration.to_csv('crime_incarceration.csv', index=False)